In [22]:
import numpy as np
import pandas as pd
import networkx as nx

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print('Set Complete')

/kaggle/input/linkpredictiondata/predict.csv
/kaggle/input/linkpredictiondata/ans_example.csv
/kaggle/input/linkpredictiondata/data_train_edge.csv
Set Complete


In [23]:
data_train_edge=pd.read_csv('/kaggle/input/linkpredictiondata/data_train_edge.csv')
predict_edge=pd.read_csv('/kaggle/input/linkpredictiondata/predict.csv')

In [24]:
predict_node1=predict_edge.node1.tolist()
predict_node2=predict_edge.node2.tolist()

In [25]:
node1_list=data_train_edge.node1.tolist()
node2_list=data_train_edge.node2.tolist()
node_number=list(set(node1_list)|set(node2_list))

In [26]:
G = nx.DiGraph()
G.add_nodes_from(node_number)

In [27]:
node_list=[]
for i in range(len(node2_list)):
    node_tuple=(node1_list[i],node2_list[i])
    node_list.append(node_tuple)

G.add_edges_from(node_list)

In [39]:
"""CN"""
def common_neighbors_score(G,u,v):
    return len(set(G.successors(u)) & set(G.successors(v)))

def common_neighbors(G,node1_list,node2_list,output_list):
    
    for i in range(len(node1_list)):
        node1=node1_list[i]
        node2=node2_list[i]
        if (node1 in node_number) and (node2 in node_number):
            length=common_neighbors_score(G,node1,node2)
            output_list.append(length)
        else:
            output_list.append(0)

"""JC"""
def Jaccard_coeffieient_score(G,u,v):
    intersection_length=len(set(G.successors(u)) & set(G.successors(v)))
    union_length=len(set(G.successors(u)) | set(G.successors(v)))
    if intersection_length is 0:
        return 0
    else:
         return float(intersection_length/union_length)

def Jaccard_coeffieient(G,node1_list,node2_list,output_list):
    
    for i in range(len(node1_list)):
        node1=node1_list[i]
        node2=node2_list[i]
        if (node1 in node_number) and (node2 in node_number):
            length=Jaccard_coeffieient_score(G,node1,node2)
            output_list.append(length)
        else:
            output_list.append(0)

"""PA1"""
def preferential_attachment_score1(G,u,v):
    return G.out_degree(u)*G.out_degree(v)
    
def preferential_attachment_1(G,node1_list,node2_list,output_list):
    
    for i in range(len(node1_list)):
        node1=node1_list[i]
        node2=node2_list[i]
        if (node1 in node_number) and (node2 in node_number):
            length=preferential_attachment_score1(G,node1,node2)
            output_list.append(length)
        else:
            output_list.append(0)

"""PA2"""
def preferential_attachment_score2(G,u,v):
    return G.out_degree(u)+G.out_degree(v)

def preferential_attachment_2(G,node1_list,node2_list,output_list):
    
    for i in range(len(node1_list)):
        node1=node1_list[i]
        node2=node2_list[i]
        if (node1 in node_number) and (node2 in node_number):
            length=preferential_attachment_score2(G,node1,node2)
            output_list.append(length)
        else:
            output_list.append(0)

"""Ad"""
def Adamic_score(G,u,v):
    cn=set(G.successors(u)) & set(G.successors(v))
    total=0
    for i in range(len(cn)):
        degree=G.out_degree(cn.pop())
        if degree is 0 or degree is 1: 
            total+=1
            continue
        log_degree=np.log(degree)
        total+=1/log_degree
    
    return total
        
def Adamic(G,node1_list,node2_list,output_list):
    
    for i in range(len(node1_list)):
        node1=node1_list[i]
        node2=node2_list[i]
        if (node1 in node_number) and (node2 in node_number):
            length=Adamic_score(G,node1,node2)
            output_list.append(length)
        else:
            output_list.append(0)

"""RA"""
def resource_allocation_score(G,u,v):
    cn=set(G.successors(u)) & set(G.successors(v))
    total=0
    for i in range(len(cn)):
        degree=G.out_degree(cn.pop())
        if degree is 0: 
            total+=1
            continue
        total+=1/degree
    
    return total
        
def resource_allocation(G,node1_list,node2_list,output_list):
    
    for i in range(len(node1_list)):
        node1=node1_list[i]
        node2=node2_list[i]
        if (node1 in node_number) and (node2 in node_number):
            score=resource_allocation_score(G,node1,node2)
            output_list.append(score)
        else:
            output_list.append(0)            


"""Katz_2"""
def Katz_2(G,node1_list,node2_list,output_list):
    for i in range(len(node1_list)):
        node1=node1_list[i]
        node2=node2_list[i]
        
        if (node1 in node_number) and (node2 in node_number):
            paths = list(nx.all_simple_paths(G, node1, node2,cutoff=2))

            length=[0,0,0]
            for ele in paths:
                length[len(ele)-1]+=1

            total=0
            for i in range(len(length)):
                if (i is not 0) and (i is not 1):
                    total+=length[i]

            output_list.append(total)
        else:
            output_list.append(0)


"""hitting time"""
def hitting_time(G,node1_list,node2_list,output_list):
    
    for i in range(len(node1_list)):
        node1=node1_list[i]
        node2=node2_list[i]
        step=5
        
        if (node1 in node_number) and (node2 in node_number):      
            score=0
            cnt=0
            actual_step=0
            paths=nx.shortest_simple_paths(G, source=node1, target=node2)
            
            for i in range(step):
                try:
                    path_now=next(paths)
                    path_length=-len(path_now)
                    actual_step+=1
                    score+=path_length
                except nx.NetworkXNoPath:
                    break
                except StopIteration:
                    break
            
            if actual_step is 0:
                output_list.append(0)
            else:
                output_list.append(score/actual_step)

        else:
            output_list.append(0)
            
    min_path=min(output_list)
    for i in range(len(output_list)):
        if output_list[i] is 0:
            output_list[i]=min_path-1

"""exit"""

'exit'

In [40]:
CN_p,JC_p,PA1_p,PA2_p,Ad_p,RA_p,Ka_p,HT_p=[],[],[],[],[],[],[],[]
column_name=['CN','JC','PA1','PA2','Ad','RA','Ka','HT']

common_neighbors(G,predict_node1,predict_node2,CN_p)
Jaccard_coeffieient(G,predict_node1,predict_node2,JC_p)
preferential_attachment_1(G,predict_node1,predict_node2,PA1_p)
preferential_attachment_2(G,predict_node1,predict_node2,PA2_p)
Adamic(G,predict_node1,predict_node2,Ad_p)
resource_allocation(G,predict_node1,predict_node2,RA_p)
Katz_2(G,predict_node1,predict_node2,Ka_p)
hitting_time(G,predict_node1,predict_node2,HT_p)

df_p=pd.DataFrame(list(zip(CN_p,JC_p,PA1_p,PA2_p,Ad_p,RA_p,Ka_p,HT_p)),columns=column_name)
df_p

,CN,JC,PA1,PA2,Ad,RA,Ka,HT
0,3,0.062500,630,51,0.648404,0.029525,2,-3.6
1,11,0.407407,357,38,3.706859,0.620308,10,-3.0
2,0,0.000000,18,11,0.000000,0.000000,0,-5.0
3,3,0.041096,1443,76,1.899689,0.525193,13,-3.0
4,0,0.000000,0,7,0.000000,0.000000,0,-4.0
...,...,...,...,...,...,...,...,...
10226,0,0.000000,13,14,0.000000,0.000000,0,-8.0
10227,0,0.000000,0,10,0.000000,0.000000,0,-8.0
10228,6,0.230769,247,32,1.727654,0.194681,3,-3.4
10229,0,0.000000,146,75,0.000000,0.000000,0,-4.0


In [41]:
"""label_top"""
def label_top_fifty(column,label):
    score=[]
    if type(column) is not type(score):
        score=column.tolist()
    else:
        score=column
        
    idx_list = sorted(range(len(score)), key = lambda k: score[k])

    for i in range(len(score)):
        label.append(1)

    for i in range(round(len(score)/2)):
        index=idx_list[i]
        label[index]=0


CN,JC,PA1,PA2,Ad,RA,Ka,HT=[],[],[],[],[],[],[],[]

label_top_fifty(df_p.CN,CN)
label_top_fifty(df_p.JC,JC)
label_top_fifty(df_p.Ad,Ad)
label_top_fifty(df_p.RA,RA)
label_top_fifty(df_p.PA1,PA1)
label_top_fifty(df_p.PA2,PA2)
label_top_fifty(df_p.Ka,Ka)
label_top_fifty(df_p.HT,HT)

sum(HT)

5115

In [42]:
"""vote_and_combine_phase1_1"""
phase1_1=[]
score=[]
for i in range(len(predict_node1)):
    score.append(CN[i]+JC[i])
    
mean=sum(score)/len(score)

for i in range(len(predict_node1)):
    if score[i]>=mean:
        phase1_1.append(1)
    else:
        phase1_1.append(0)
        
print(mean)
print(sum(phase1_1))

0.999902257843808
5376


In [50]:
"""vote_and_combine_phase1_2"""
phase1_2=[]
score=[]
for i in range(len(predict_node1)):
    score.append(PA1[i]+PA2[i])
    
mean=sum(score)/len(score)

for i in range(len(predict_node1)):
    if score[i]>=mean:
        phase1_2.append(1)
    else:
        phase1_2.append(0)
        
print(mean)
print(sum(phase1_2))

0.999902257843808
5664


In [44]:
"""phase1_3"""
phase1_3=[]
score=[]
for i in range(len(predict_node1)):
    score.append(RA[i]+Ad[i])
    
mean=sum(score)/len(score)

for i in range(len(predict_node1)):
    if score[i]>=mean:
        phase1_3.append(1)
    else:
        phase1_3.append(0)

print(mean)
print(sum(phase1_3))

0.999902257843808
5266


In [45]:
"""phase2"""
phase2=[]
score=[]
for i in range(len(predict_node1)):
    score.append(phase1_1[i]+phase1_2[i])
    
mean=sum(score)/len(score)

for i in range(len(predict_node1)):
    if score[i]>=mean:
        phase2.append(1)
    else:
        phase2.append(0)
        
print(mean)
print(sum(phase2))

1.0790734043593002
4475


In [46]:
"""phase3"""
phase3=[]
score=[]
for i in range(len(predict_node1)):
    score.append(phase1_3[i]+phase2[i])
    
mean=sum(score)/len(score)

for i in range(len(predict_node1)):
    if score[i]>=mean:
        phase3.append(1)
    else:
        phase3.append(0)

print(mean)
print(sum(phase3))

0.9521063434659368
5310


In [47]:
"""phase4"""
phase4=[]
score=[]
for i in range(len(predict_node1)):
    score.append(phase3[i]+Ka[i])
    
mean=sum(score)/len(score)

for i in range(len(predict_node1)):
    if score[i]>=mean:
        phase4.append(1)
    else:
        phase4.append(0)
        
print(mean)
print(sum(phase4))

1.0189619783012414
4744


In [48]:
"""phase5"""
phase5=[]
score=[]
for i in range(len(predict_node1)):
    score.append(phase4[i]+HT[i])
    
mean=sum(score)/len(score)

for i in range(len(predict_node1)):
    if score[i]>=mean:
        phase5.append(1)
    else:
        phase5.append(0)
        
print(mean)
print(sum(phase5))

0.9636399178965888
5203


In [49]:
ans=[]
for i in range(len(predict_node1)):
    ans.append(phase5[i])
sum(ans)

5203

In [20]:
df_ans=pd.DataFrame(ans,columns=['ans'])
df_ans.index.name='predict_nodepair_id'

In [21]:
cnt=0
for i in range(len(df_ans)):
    if df_ans.ans[i]==1:
        cnt+=1
print(cnt)

5203


In [ ]:
df_ans.to_csv('answer.csv',index=True)